<a href="https://colab.research.google.com/github/sanazy/Persian-Car-Licence-Plate-Detection-and-Recognition/blob/main/Separate_Plates_from_Car_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
PATH = '/content/gdrive/MyDrive/DPA/'
YOLO_PATH = PATH + 'yolov7'
DATA_PATH = PATH + 'cars'
save_path = PATH + 'my_plates'
weights_path = PATH + "my_weights/best.pt"

In [3]:
%cd {PATH}/yolov7

/content/gdrive/MyDrive/DPA/yolov7


In [4]:
import os
from pathlib import Path
from typing import Union
import torch
import cv2 as cv
import numpy as np
from models.experimental import attempt_load
from utils.general import check_img_size
from utils.torch_utils import select_device, TracedModel
from utils.datasets import letterbox
from utils.general import non_max_suppression, scale_coords
from utils.plots import plot_one_box, plot_one_box_PIL
from copy import deepcopy

import keras
import cv2
import random
import matplotlib.pyplot as plt
import seaborn as sn
import tensorflow as tf

import argparse
from keras.models import load_model
import functools

from google.colab.patches import cv2_imshow

In [5]:
device_id = 'cpu'
image_size = 640
trace = True

# Initialize
device = select_device(device_id)
half = device.type != 'cpu'  # half precision only supported on CUDA

# Load model
model = attempt_load(weights_path, map_location=device)  # load FP32 model
stride = int(model.stride.max())  # model stride
imgsz = check_img_size(image_size, s=stride)  # check img_size

if trace:
  model = TracedModel(model, device, image_size)

if half:
  model.half()  # to FP16
    
if device.type != 'cpu':
  model(torch.zeros(1, 3, imgsz, imgsz).to(device).type_as(next(model.parameters())))  # run once

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 



In [6]:
def detect_plate(source_image):
  # Padded resize
  img_size = 640
  stride = 32
  img = letterbox(source_image, img_size, stride=stride)[0]

  # Convert
  img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x416
  img = np.ascontiguousarray(img)
  img = torch.from_numpy(img).to(device)
  img = img.half() if half else img.float()  # uint8 to fp16/32
  img /= 255.0  # 0 - 255 to 0.0 - 1.0
  if img.ndimension() == 3:
    img = img.unsqueeze(0)
      
  with torch.no_grad():
    # Inference
    pred = model(img, augment=True)[0]

  # Apply NMS
  pred = non_max_suppression(pred, 0.25, 0.45, classes=0, agnostic=True)

  plate_detections = []
  det_confidences = []
  
  # Process detections
  for i, det in enumerate(pred):  # detections per image
    if len(det):
      # Rescale boxes from img_size to im0 size
      det[:, :4] = scale_coords(img.shape[2:], det[:, :4], source_image.shape).round()

      # Return results
      for *xyxy, conf, cls in reversed(det):
        coords = [int(position) for position in (torch.tensor(xyxy).view(1, 4)).tolist()[0]]
        plate_detections.append(coords)
        det_confidences.append(conf.item())

  return plate_detections, det_confidences

In [7]:
def crop(image, coord):
  cropped_image = image[int(coord[1]):int(coord[3]), int(coord[0]):int(coord[2])]
  return cropped_image

In [8]:
for image_name in os.listdir(DATA_PATH):
  image_path = os.path.join(DATA_PATH, image_name)
  plate_image = cv.imread(image_path)
  
  plate_detections, _ = detect_plate(plate_image)
  for coords in plate_detections:
    plate_region = crop(plate_image, coords)
    cv.imwrite(os.path.join(save_path, "plate_" + image_name), plate_region)

/usr/local/lib/python3.8/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
